# **Particle Swarm optimization**
class Particle:
- Define the particle object with the **__init__**. Every particle must have a position, a velocity.
- Define a function, called **FitnessCalculator**, that computes the fitness value of the particle. Its indipendent variable is the **accuracy metric** of the neural network.
- Define a function, called **PositionCalculator**
- Define a function, called **VelocityCalculator**


In [28]:
import numpy as np
import random
from particle import Particle

In [29]:
def InitializeSwarm(swarm_size, dimensionality, lower_bound, upper_bound):
    '''It takes as input the swarm size (number of particles I want to create, the dimensionality of the swarm (number of parameter for each particle) and the lower and upper bound of the parameter (that are two lists))'''

    #np.random.seed(3)
    # Usually the positions of particles are initialized to uniformly cover the search space
    swarm_list = []
    for particle in range(swarm_size):
        position = np.random.uniform(lower_bound, upper_bound, dimensionality)

        velocity = np.zeros(dimensionality)
        # velocity = np.random.random(dimensionality)

        part = Particle(position, velocity)
        swarm_list.append(part)

    return swarm_list

In [30]:
# Let's first define the function we want to use for the evaluation:

def f(lista):
    '''Definisco una funzione f che prende in input una lista di tre elementi (ovviamente va cambiata se cambio la swarm size)'''
    return(lista[0]**2+lista[1]**2+lista[2]**2)

In [31]:
def global_optimum(swarm, problem = 'minimum'):
    '''takes as input a swarm of particles and return the position of global optimum and the value of the fitness function in that global optimum'''

    if problem == 'minimum':
        global_opt = (min([particle.bestfit for particle in swarm]))
        best_global_position = swarm[np.argmin(np.array([particle.bestfit for particle in swarm]))].position
        
    elif problem == 'maximum':
        global_opt = (max([particle.bestfit for particle in swarm]))
        best_global_position = swarm[np.argmax(np.array([particle.bestfit for particle in swarm]))].position

    return best_global_position, global_opt  

In [32]:
def acceleration_coefficient(iteration, total_iterations):

    c1_min = 0.5
    c1_max = 2.5
    c2_min = 0.5
    c2_max = 2.5

    c1 = (c1_min-c1_max)*(iteration/total_iterations) + c1_max
    c2 = (c2_max-c2_min)*(iteration/total_iterations) + c2_min
    return c1,c2


In [33]:
def PSO_inizialization(swarm_size, dim, evaluation_funct, lower_bound, upper_bound, problem):
    swarm = InitializeSwarm(swarm_size, dim, lower_bound, upper_bound)

    #First we need to evaluate each particle

    for particle in swarm:
        particle.FitnessCalculator(particle.position, evaluation_funct)

        # When we inizialize the swarm we need also to calculate the local optimum:
        particle.BestLocal(problem)

    return(swarm)

In [34]:
def PSO_alg(swarm, lower_bound, upper_bound, v_max, evaluation_funct, max_iteration, problem):

    # First we calculate the starting global best position of the swarm
    best_global_position, global_opt = global_optimum(swarm, problem)

    # Calculate the coefficient needed for updating the velocity of the particles.
    c1, c2 = acceleration_coefficient(swarm[0].iteration, max_iteration)

    # set the termination criteria
    criteria_not_reach = True

    while criteria_not_reach:

        for particle in swarm:
            ## We need to calculate the new velocity:",
            particle.VelocityCalculator(c1, c2, best_global_position, w_schedule = 'nonlinearly decreasing', v_max = v_max)

            ## We can calculate the new position:",
            particle.PositionCalculator(lower_bound, upper_bound, evaluation_funct, problem)

            #print('New velocity: ', particle.velocity, '\n New position: ', particle.position, '\n New fitness: ', particle.fitness , '\n Best local position: ', particle.bestp, 'Best local fitness', particle.bestfit ,'\n')


        # Then we update the global optimum
        best_position, opt = global_optimum(swarm, problem)

        if problem == 'minimum':
            if opt < global_opt:
                global_opt = opt
                best_global_position = best_position
        elif problem == 'maximum':
            if opt > global_opt:
                global_opt = opt
                best_global_position = best_position
    
        
        # print('GLOBAL OPTIMUM: ', global_opt, 'GLOBAL OPT POSITION: ', best_global_position)

        # we set the criteria depending on the number of iteration
        if swarm[0].iteration == max_iteration:
            criteria_not_reach = False
    
    return best_global_position, global_opt

In [35]:
def main():
    swarm_size = 100
    dim = 3
    evaluation_funct = f
    lower_bound=[0,0,0]
    upper_bound = [10,10,10]
    v_max = [3,3,3]
    problem = 'minimum'
    max_iteration = 200


    swarm = PSO_inizialization(swarm_size, dim, evaluation_funct, lower_bound, upper_bound, problem)

    best_global_position, global_opt = PSO_alg(swarm, lower_bound, upper_bound, v_max, evaluation_funct, max_iteration, problem)
    
    print('Best position found: ', best_global_position, 'with an evaluation of: ',global_opt)

In [36]:
main()

Best position found:  [0.25188981 0.06491951 0.04260709] with an evaluation of:  0.06947838350014433
